In [63]:
df = pd.read_csv("./data/total_data.csv")

In [46]:
df["시놉시스"]

0        크리스마스 시즌의 시카고. 말썽꾸러기 케빈(맥컬리 컬킨)은 자기의 치즈 피자를 먹은...
1        클라크 오스왈드(Clark Wilhelm Griswold, Jr.: 체비 체이스 분...
2        초심리를 연구하는 뉴욕의 괴짜교수 피터 밴크맨(빌 머레이)와 레이몬드 스탠드(댄 애...
3        고스트 버스터즈 4인방은 악마 '고저'로부터 세계를 구한 후 순탄한 생활을 하지 못...
4        조지 베일리는 베드폴드 마을 사람들을 위해 평생을 봉사하며 살아온 인물. 그는 항상...
                               ...                        
19139                    악을 징벌하는 대영웅으로 자라나는 특출한 소년 목진의 성장기
19140    영화에서는 잘만 풀리는 로맨스, 현실에선 왜 이리 어려울까? 도대체 ‘사랑이 뭔데!...
19141                            일본의 디바 탄생에 숨겨진 사랑과 이별 이야기
19142    여름, 미국 태평양 연안 북서부의 바다가 금광으로 변신한다! 값비싼 참치 떼를 잡기...
19143                         수수께끼의 재해인 세계 동시 도시 소멸로부터 5년.
Name: 시놉시스, Length: 19144, dtype: object

In [48]:
import konlpy
from konlpy.tag import Okt
okt = Okt()

import re
def sub_special(s):
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',s)
STOP_WORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
def morph_and_stopword(s):
    token_ls = []
    #형태소 분석
    tmp = okt.morphs(s, stem=True)

    #불용어 처리
    for token in tmp:
        if token not in STOP_WORDS:
            token_ls.append(token)
    return token_ls

df["시놉시스"] = df["시놉시스"].apply(sub_special).apply(morph_and_stopword)
df["시놉시스"] = df["시놉시스"].apply(lambda x: " ".join(x))

In [49]:
# 
df["시놉시스"][0: 2]

0    크리스마스 시즌의 시카고. 말썽꾸러기 케빈(맥컬리 컬킨)은 자기의 치즈 피자를 먹은...
1    클라크 오스왈드(Clark Wilhelm Griswold, Jr.: 체비 체이스 분...
Name: 시놉시스, dtype: object

In [50]:
df["시놉시스"][0: 2].apply(sub_special).apply(morph_and_stopword)

0    [크리스마스, 시즌, 시카고, 말썽, 꾸러기, 케빈, 맥, 컬리, 컬킨, 자기, 치...
1    [클라크, 오스왈드, Clark, Wilhelm, Griswold, Jr, 체비, ...
Name: 시놉시스, dtype: object

In [64]:
# df = pd.read_csv("./data/total_data.csv")

df["장르"][df["장르"].isna()] = "['']"  # 이후에는 제거
df["장르"] = df["장르"].apply(lambda x: eval(x))
df["장르"] = df["장르"].apply(lambda x : (' ').join(x))

# isna_index = df["평점"].isna()
# df["평점"][isna_index] = "['']"
# df["평점"] = df["평점"].apply(lambda x : eval(x))

# def print_ls_1(x):
#     if len(x) == 1:
#         return 0
#     else:
#         return x[1].split()[0]
    
# def print_ls_2(x):
#     if len(x) == 1:
#         return "0k"
#     else:
#         try:
#             return x[1].split()[1].strip("()")
#         except:
#             return "0k"
        
# df["평가수"] = df["평점"].apply(print_ls_2)
# df["평점"] = df["평점"].apply(print_ls_1)
# df["평가수"] = df["평가수"].apply(lambda x: str(int(x[:-1]) * 1000) if x[-1] == "m" else x[:-1])
df["평점"] = df["평점"].astype("float")
df["평가수"] = df["평가수"].astype("int")

C = df["평점"].mean()
m = df["평가수"].quantile(0.6)
def weighted_vote_average(record):
    v = record["평가수"]
    R = record["평점"]
    
    return ((v/(v+m)) * R) + ((m/(m+v)) * C)
df["가중치평점"] = df.apply(weighted_vote_average, axis=1)

from ast import literal_eval
def literal(x):
    return literal_eval(x)

def sum_list(x):
    ls = []
    for i in x[:10]:
        text = i.strip()
        text = text.replace('.', '')
        text = text.replace("'", '_')
        text = text.replace(" ", '_')
        ls.append(text)
    return " ".join(ls)

df["감독"][df["감독"].isna()] = "['']"
df["감독"] = df["감독"].apply(literal)
df["감독"] = df["감독"].apply(sum_list)

df["출연진"][df["출연진"].isna()] = "['']"
df["출연진"] = df["출연진"].apply(lambda x: eval(x))
df["출연진"] = df["출연진"].apply(sum_list)

df["특징"] = df["장르"] + " " + df["종류"] + " " + df["감독"] + " " + df["출연진"] + " " + df["시놉시스"]

In [3]:
# # title_list = ["나 홀로 집에", "해리 포터와 마법사의 돌", "쇼생크 탈출", "브레이킹 배드"]
# title_list = ["해리 포터와 마법사의 돌", "아케인", "쇼생크 탈출"]
# # title_list = titles
# sum_find = " ".join(list(df[df["제목"].apply(lambda x : x in title_list)]["특징"]))

# title_indexes = []
# for title_name in title_list:
#     title_movie = df[df["제목"] == title_name]
#     title_indexes += list(title_movie.index.values)

<19144x282925 sparse matrix of type '<class 'numpy.float64'>'
	with 1250678 stored elements in Compressed Sparse Row format>

array([[    0,    13,  2686, ..., 15854, 15857, 19143],
       [    1,  1672,  6963, ..., 18611, 18609, 19143],
       [    2,     3,  2181, ..., 18627, 16287, 15663],
       ...,
       [19141, 16530, 10924, ...,  9991,  9990,     0],
       [19142, 18443, 16469, ..., 13368,  7802,  9571],
       [19143, 15559, 17306, ...,  9837,  9836,     0]], dtype=int64)

# CountVectorizer

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
# count_vect = CountVectorizer(min_df=0)
genre_mat = count_vect.fit_transform(df["특징"])

In [5]:
genre_mat.shape
# (19144, 238365)

(19144, 238365)

In [6]:
genre_sim = cosine_similarity(genre_mat, genre_mat)
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]

In [7]:
genre_sim_sorted_ind

array([[    0,    13, 13058, ..., 15471, 15470, 19143],
       [    1,  3184,  3615, ..., 17376, 17377, 19143],
       [    2,     3, 12016, ..., 18431, 18800, 17110],
       ...,
       [19141, 19125, 19113, ..., 10120, 10119,     0],
       [19046, 19020, 19088, ..., 10223, 10222,     0],
       [19143, 19041, 19063, ...,  9952,  9950,     0]], dtype=int64)

In [8]:

temp = pd.DataFrame(genre_sim_sorted_ind[:, :21])
temp.to_csv("./data/genre_sim_matrix.csv", index=False)

In [9]:
temp = pd.DataFrame(genre_sim_sorted_ind[:, :21])
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19144 entries, 0 to 19143
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       19144 non-null  int64
 1   1       19144 non-null  int64
 2   2       19144 non-null  int64
 3   3       19144 non-null  int64
 4   4       19144 non-null  int64
 5   5       19144 non-null  int64
 6   6       19144 non-null  int64
 7   7       19144 non-null  int64
 8   8       19144 non-null  int64
 9   9       19144 non-null  int64
 10  10      19144 non-null  int64
 11  11      19144 non-null  int64
 12  12      19144 non-null  int64
 13  13      19144 non-null  int64
 14  14      19144 non-null  int64
 15  15      19144 non-null  int64
 16  16      19144 non-null  int64
 17  17      19144 non-null  int64
 18  18      19144 non-null  int64
 19  19      19144 non-null  int64
 20  20      19144 non-null  int64
dtypes: int64(21)
memory usage: 3.1 MB


In [27]:
# for i in range(len(temp)):
#     temp.iloc[i:i+1].to_csv(f"./data/genre_sim_matrix/{i}.csv", index=False)

In [148]:
## title 의 id(index) 를 찾는 코드
ind = df[df.제목 == "주술회전"].index[0]

In [150]:
similar_indexes = pd.read_csv(f"./data/genre_sim_matrix/{ind}.csv")
similar_indexes = np.array(similar_indexes)[:,:20 + 1]

similar_indexes = similar_indexes.reshape(-1)
similar_indexes = similar_indexes[similar_indexes != ind]  # 타이틀 기준 제거

In [152]:
df.iloc[similar_indexes].sort_values("가중치평점", ascending=False)[:20][["제목", "가중치평점"]]

,제목,가중치평점
17205,귀멸의 칼날,8.521266
15606,코드 기아스 반역의 를르슈,8.512476
17268,할리 퀸,8.056709
17258,스타워즈: 배드 배치,7.667731
17250,도타: 용의 피,7.442278
17209,What If...?,7.415962
17382,키포와 신기한 동물들,7.020126
17434,우주 히어로 키드,6.252202
17405,소드 아트 온라인 얼터너티브: 건 게일 온라인,6.160169
16433,육화의 용사,6.052202


In [94]:
temp

,0,1,2,3,4,5,6,7,8,9,...,19134,19135,19136,19137,19138,19139,19140,19141,19142,19143
0,0,13,13058,7302,8626,12691,7555,1505,8064,13031,...,15489,18381,15484,15483,15482,15477,15474,15471,15470,19143
1,1,3184,3615,3632,1885,1443,6093,6059,2495,2811,...,17367,17368,17369,17370,17371,17372,17374,17376,17377,19143
2,2,3,12016,8654,13100,11002,3146,10709,8676,11497,...,18803,18430,15425,18802,16658,15805,18801,18431,18800,17110
3,3,2,10993,5723,15168,17492,17423,11002,1390,14064,...,18687,18685,18684,16237,18682,16241,16243,18681,16245,18308
4,4,6989,453,12328,819,7337,10883,6934,14995,7687,...,17883,17892,17910,17893,17896,17899,17905,17907,17908,19143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19139,19139,19063,19041,18668,17385,19003,19083,15934,19090,17388,...,9970,9968,9967,9966,9965,9964,9963,9962,9961,0
19140,19140,18999,19072,19101,15387,15969,19088,19087,16247,19085,...,10228,10227,10226,10225,10224,10223,10222,10221,10220,0
19141,19141,19125,19113,19098,19082,19101,15969,18987,19074,19000,...,10129,10127,10126,10125,10123,10122,10121,10120,10119,0
19142,19046,19020,19088,19101,19102,19087,19085,19074,19072,19039,...,10230,10229,10228,10227,10226,10225,10224,10223,10222,0


In [157]:
temp.iloc[:, :20]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,13,13058,7302,8626,12691,7555,1505,8064,13031,7434,12808,8552,137,10846,7679,6621,7405,12813,8997
1,1,3184,3615,3632,1885,1443,6093,6059,2495,2811,6672,2472,1057,4481,6100,6165,2697,2971,5658,4271
2,2,3,12016,8654,13100,11002,3146,10709,8676,11497,11986,16669,11574,8504,14064,10993,10834,17095,15879,9525
3,3,2,10993,5723,15168,17492,17423,11002,1390,14064,17095,10834,12016,12905,11015,6595,11361,4299,11831,7998
4,4,6989,453,12328,819,7337,10883,6934,14995,7687,8552,12808,18818,6054,10944,12623,8301,240,5578,8653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19139,19139,19063,19041,18668,17385,19003,19083,15934,19090,17388,17492,17734,17190,18065,18868,18919,19069,19027,17973,16470
19140,19140,18999,19072,19101,15387,15969,19088,19087,16247,19085,19020,18987,19039,15757,19074,19102,19000,19046,19142,19133
19141,19141,19125,19113,19098,19082,19101,15969,18987,19074,19000,19102,18999,15387,19039,19088,19087,19085,19046,19020,19072
19142,19046,19020,19088,19101,19102,19087,19085,19074,19072,19039,15387,16247,15757,18999,19142,18987,15969,19000,19133,15873


In [82]:
# ' . 으로 단어 구분하는거 삭제하기

In [112]:
# df[["제목", "가중치평점"]].to_csv("./data/title_evaluation.csv", index=False)

In [145]:
## 제목, 가중치평점만 있는 데이터
df = pd.read_csv("./data/title_evaluation.csv")

## title 의 id(index) 를 찾는 코드
ind = df[df.제목 == "주술회전"].index[0]

In [146]:
## 특징 유사도 매트릭스
similar_indexes = pd.read_csv(f"./data/genre_sim_matrix.csv")
similar_indexes = np.array(similar_indexes)[ind,:]
similar_indexes = similar_indexes.reshape(-1)
similar_indexes = similar_indexes[similar_indexes != ind]  # 타이틀 기준 제거

In [147]:
df.iloc[similar_indexes].sort_values("가중치평점", ascending=False)[:20][["제목", "가중치평점"]]

,제목,가중치평점
17205,귀멸의 칼날,8.521266
15606,코드 기아스 반역의 를르슈,8.512476
17268,할리 퀸,8.056709
17258,스타워즈: 배드 배치,7.667731
17250,도타: 용의 피,7.442278
17209,What If...?,7.415962
17382,키포와 신기한 동물들,7.020126
17434,우주 히어로 키드,6.252202
17405,소드 아트 온라인 얼터너티브: 건 게일 온라인,6.160169
16433,육화의 용사,6.052202


# csv 형태

In [143]:
pd.read_csv("./data/title_evaluation.csv")

,제목,가중치평점
0,나 홀로 집에,7.586332
1,크리스마스 대소동,7.560676
2,고스트 버스터즈,7.779207
3,고스트 버스터즈 2,6.584805
4,멋진 인생,8.574506
...,...,...
19139,대주재,5.840253
19140,카란 조하르의 사랑이 뭔데!,5.840253
19141,M 사랑해야 할 사람이 있어서,5.840253
19142,배틀피시,5.840253


In [144]:
pd.read_csv(f"./data/genre_sim_matrix.csv")

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0,13,13058,7302,8626,12691,7555,1505,8064,13031,...,12808,8552,137,10846,7679,6621,7405,12813,8997,13123
1,1,3184,3615,3632,1885,1443,6093,6059,2495,2811,...,2472,1057,4481,6100,6165,2697,2971,5658,4271,3201
2,2,3,12016,8654,13100,11002,3146,10709,8676,11497,...,16669,11574,8504,14064,10993,10834,17095,15879,9525,10722
3,3,2,10993,5723,15168,17492,17423,11002,1390,14064,...,10834,12016,12905,11015,6595,11361,4299,11831,7998,6179
4,4,6989,453,12328,819,7337,10883,6934,14995,7687,...,12808,18818,6054,10944,12623,8301,240,5578,8653,4676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19139,19139,19063,19041,18668,17385,19003,19083,15934,19090,17388,...,17734,17190,18065,18868,18919,19069,19027,17973,16470,18491
19140,19140,18999,19072,19101,15387,15969,19088,19087,16247,19085,...,18987,19039,15757,19074,19102,19000,19046,19142,19133,18532
19141,19141,19125,19113,19098,19082,19101,15969,18987,19074,19000,...,18999,15387,19039,19088,19087,19085,19046,19020,19072,15757
19142,19046,19020,19088,19101,19102,19087,19085,19074,19072,19039,...,16247,15757,18999,19142,18987,15969,19000,19133,15873,18511


# 여러 타이틀이 들어올 경우 코드

In [163]:

titles = ["What If...?", "귀멸의 칼날", "대부", "아케인"]

In [164]:
import pandas as pd
import numpy as np

## 제목, 가중치평점만 있는 데이터
df = pd.read_csv("./data/title_evaluation.csv")

## title 의 id(index) 를 찾는 코드
indexes = []
for title in titles:
    indexes.append(df[df.제목 == title].index[0])
indexes

[17209, 17205, 12, 17208]

In [165]:
## 특징 유사도 매트릭스
similar_indexes = pd.read_csv(f"./data/genre_sim_matrix.csv")
similar_indexes = np.array(similar_indexes)[indexes,:]
similar_indexes = similar_indexes.reshape(-1)
similar_indexes = set(similar_indexes)
similar_indexes = np.array(list(similar_indexes))
for i in indexes:
    similar_indexes = similar_indexes[similar_indexes != i]  # 타이틀 기준 제거

In [166]:
df.iloc[similar_indexes].sort_values("가중치평점", ascending=False)[:20][["제목", "가중치평점"]]

,제목,가중치평점
88,대부 2,8.987411
15606,코드 기아스 반역의 를르슈,8.512476
17280,"러브, 데스 + 로봇",8.416228
17216,주술회전,8.406693
122,지옥의 묵시록,8.383825
16871,캐슬바니아,8.138705
17268,할리 퀸,8.056709
11100,귀멸의 칼날 극장판 무한열차편,7.990245
17290,파이널 스페이스,7.985478
6101,어머니,7.675572


In [43]:
import Recommendation2

result = Recommendation2.recommendations(["스파이더맨"])
result

,제목,가중치평점
1659,스파이더맨 2,7.290252
15654,페이트 / 스테이 나이트,6.651223
8710,배트맨 대 슈퍼맨: 저스티스의 시작,6.396658
9877,스페이스 비트윈 어스,6.359291
3287,스파이더맨 3,6.197360
16791,청운지,5.840253
11002,Satria Heroes: Revenge of the Darkness,5.840253
17528,Dollar,5.840253
7921,산이 울다,5.840253
14064,솔 레반테,5.840253


In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.특징)
genre_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
genre_sim_sorted_ind

array([[    0,    13,  2686, ..., 15854, 15857, 19143],
       [    1,  1672,  6963, ..., 18611, 18609, 19143],
       [    2,     3,  2181, ..., 18627, 16287, 15663],
       ...,
       [19141, 16530, 10924, ...,  9991,  9990,     0],
       [19142, 18443, 16469, ..., 13368,  7802,  9571],
       [19143, 15559, 17306, ...,  9837,  9836,     0]], dtype=int64)

In [83]:
q = "바이올렛"
df[df["제목"].apply(lambda x : q in x)]["제목"]

3995                      울트라바이올렛
7283              바이올렛: 그녀의 뜨거운 삶
12179               극장판 바이올렛 에버가든
12336    바이올렛 에버가든 외전: 영원과 자동수기인형
17035                    울트라 바이올렛
17624                   바이올렛 에버가든
Name: 제목, dtype: object

In [66]:

titles = ["스파이더맨"]

In [67]:
import pandas as pd
import numpy as np

## 제목, 가중치평점만 있는 데이터
# df = pd.read_csv("./data/title_evaluation.csv")

## title 의 id(index) 를 찾는 코드
indexes = []
for title in titles:
    indexes.append(df[df.제목 == title].index[0])
indexes

[1657]

In [68]:
## 특징 유사도 매트릭스
# similar_indexes = pd.read_csv(f"./data/genre_sim_matrix.csv")
# similar_indexes = np.array(similar_indexes)[indexes,:]
similar_indexes = genre_sim_sorted_ind[indexes, :20]
similar_indexes = similar_indexes.reshape(-1)
similar_indexes = set(similar_indexes)
similar_indexes = np.array(list(similar_indexes))
for i in indexes:
    similar_indexes = similar_indexes[similar_indexes != i]  # 타이틀 기준 제거

In [69]:
# df.iloc[similar_indexes][:20][["제목", "가중치평점"]]
df.iloc[similar_indexes][:20][["제목", "가중치평점", "시놉시스"]]

,제목,가중치평점,시놉시스
9739,스파이더맨: 홈커밍,7.388819,토니 스타크의 제안을 받고 시빌 워에 참여한 피터 파커는 다음 임무가 언제일지 설렘...
7839,팬,5.708500,"피터팬, 그 전설이 시작된다! “이야기를 하나 해줄게요. 어른이 되지 않는 아이, ..."
11039,스파이더맨: 뉴 유니버스,8.377196,평범한 10대 마일스 모랄레스는 우연히 방사능 거미에 물려 스파이더맨 능력을 가지게...
11042,스파이더맨: 파 프롬 홈,7.384557,"어벤져스 멤버들과 타노스와의 대접전을 마친 뒤, 지난 5년 동안 사라졌던 사람들이 ..."
2346,윔블던,6.272552,별 볼일 없는 테니스 선수에서 돈 많고 할 일 없는 아줌마들의 테니스 강사로 전락한...
5035,그린 랜턴: 반지의 선택,5.504843,머나먼 은하계에는 무한한 힘을 지닌 반지로 지구를 포함한 총 3600개 섹터로 구분...
16706,레고 바이오니클: 하나가 되기 위한 여정,5.840253,6명의 전설적인 영웅들이 자연의 힘을 활용해 악의 힘을 무찌르기 위한 대장정에 나선다.
7367,다크 월드: 이퀄리브리엄,5.840253,원래 다크 월드에서만 볼 수 있는 어둠의 존재 ‘섀도우’는 사람들의 에너지와 감정을...
6856,윈터스 테일,6.174755,"1895년 뉴욕에서는 폐병이 유행하였고, 피터의 아버지는 병으로 입국이 금지되자 갓..."
3534,알러뷰 맨,6.977260,사회적으로도 성공하고 사랑하는 여인과 결혼을 앞두고 있는 피터는 들러리를 서 줄 단...


In [69]:
df.iloc[similar_indexes].sort_values("가중치평점", ascending=False)[:20][["제목", "가중치평점"]]

,제목,가중치평점
1804,블레이드,7.080913
3737,언더월드: 라이칸의 반란,6.581006
11391,더 웨이 홈,6.528051
2790,제이콥의 거짓말,6.368051
5027,링컨: 뱀파이어 헌터,5.898478
11232,시스터즈 워: 레드스네이크,5.892202
11774,코드 카림,5.840253
10413,몬스터 랜드,5.840253
10701,뱀파이어 클린업 디파트먼트,5.840253
8783,언더월드: 블러드 워,5.802091


In [44]:
import Recommendation2

result = Recommendation2.recommendations(["스파이더맨"])
result

,제목,가중치평점
1659,스파이더맨 2,7.290252
15654,페이트 / 스테이 나이트,6.651223
8710,배트맨 대 슈퍼맨: 저스티스의 시작,6.396658
9877,스페이스 비트윈 어스,6.359291
3287,스파이더맨 3,6.197360
16791,청운지,5.840253
11002,Satria Heroes: Revenge of the Darkness,5.840253
17528,Dollar,5.840253
7921,산이 울다,5.840253
14064,솔 레반테,5.840253
